In [1]:
import os

In [2]:
os.chdir("../") # go to the root directory of the project

In [3]:
%pwd

'd:\\Kidney-Disease-Classification-Deep-Learning-Project'

In [34]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: 0.001

In [35]:
import os
import sys

# Get the Current Working Directory (CWD) of the notebook process
current_dir = os.getcwd() 

# Assuming your project root is the CWD, you can define 'src_dir' relative to it
# If 'src' is directly in the CWD (project_root/src)
src_dir = os.path.join(current_dir, 'src') 

# If the notebook is deep inside the project and you need to go up levels:
# current_dir = os.getcwd() 
# project_root = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir)) # Example: go up two levels
# src_dir = os.path.join(project_root, 'src') 


# Add the 'src' directory to the system path
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)



In [36]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [37]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [38]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        self.model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params.params_learning_rate),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"]
    )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [43]:
tf.config.run_functions_eagerly(True)

In [44]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-10-06 22:13:42,002: INFO: common: YAML file loaded successfully: D:\Kidney-Disease-Classification-Deep-Learning-Project\config\config.yaml]
[2025-10-06 22:13:42,009: INFO: common: YAML file loaded successfully: D:\Kidney-Disease-Classification-Deep-Learning-Project\params.yaml]
[2025-10-06 22:13:42,012: INFO: common: created directory at: artifacts]
[2025-10-06 22:13:42,015: INFO: common: created directory at: artifacts\training]


TypeError: TrainingConfig.__init__() missing 1 required positional argument: 'params_learning_rate'